In [23]:
import requests
from selenium import webdriver

from bs4 import BeautifulSoup
from string import printable
import unicodedata
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import re


base_url = "https://spb.hh.ru/resume/"

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

In [11]:
from selenium.webdriver.firefox.options import Options
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

SessionNotCreatedException: Message: Unable to find a matching set of capabilities


In [15]:
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.headless = True

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/home/royalcat/.wdm/drivers/chromedriver/linux64/85.0.4183.87/chromedriver] found in cache
 


In [16]:
def salary(content):
    soup = BeautifulSoup(content)
    text = soup.find('span', 
        {
            'class':'resume-block__salary resume-block__title-text_salary',                    "data-qa":"resume-block-salary"
        }).text
    text = unicodedata.normalize('NFKD', text)
    salary_list = [char for char in text if char.isdigit()]
    return int(''.join(salary_list))

In [54]:
def work_exp(content):
    soup = BeautifulSoup(content)
    text = soup.find('span', {
            'class':'resume-block__title-text resume-block__title-text_sub'
        }).text
    
    text = unicodedata.normalize('NFKD', text)
    return text.replace('Опыт работы', '').strip()

In [55]:
def drive_exp(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-driver-experience'
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].text
    text = unicodedata.normalize('NFKD', text)
    return text.strip()

In [56]:
def education(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-education'
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].text
    text = unicodedata.normalize('NFKD', text)
    return text.strip()

In [57]:
def employment(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-position',
            'class': 'resume-block',
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].div.div.p.text
    text = unicodedata.normalize('NFKD', text)
    
    return text.replace('Занятость:', '').strip()

In [58]:
def schedule(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-position',
            'class': 'resume-block',
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].div.div.find_all('p')[1].text
    text = unicodedata.normalize('NFKD', text)
    
    return text.replace('График работы:', '').strip()

In [59]:
term_parser = {
    'зарплата': salary,
    'зп': salary,
    'опыт работы': work_exp,
    'опыт вождения': drive_exp,
    'образование': education,
    'занятость': employment,
    'график': schedule
}

In [76]:
def parser(question_terms, uid):
    #content = download.resume(uid)
    driver.get(base_url + uid)
    content = driver.page_source
    output = {}
    for name in question_terms:
        name_term = term_parser[name](content)
        if type(name_term) is str:
            name_term = name_term.lower()
        output[name] = name_term
    return output

In [77]:
parser(['зарплата', 'опыт работы', 'опыт вождения', 'образование', 'занятость', 'график'], 'fc6280cb00051725840039ed1f574c52506830')

{'зарплата': 40000,
 'опыт работы': '9 месяцев',
 'опыт вождения': 'права категории b',
 'образование': 'среднее образование',
 'занятость': 'полная занятость',
 'график': 'полный день'}